# Описание задачи
Наши студенты задают множество вопросов в общем чате на совершенно различные темы: от общих тем и вопросов оплаты курсов, до трудоустройства и даже вопросов, не относящихся к области деятельности платформы. Для ускорения обработки запросов и снижения нагрузки на поддержку было принято решение создать Telegram-бота для автоматизации ответов и эскалации сложных запросов к живым операторам.



## CLIP: архитектура модели
В 2021 году появилась мультимодальная модель CLIP (Contrastive Language–Image Pre-training; не путать... ох, ни с чем не путать) от OpenAI. Модель обучается на большом объёме данных, сочетающих текст и изображения, и способна понимать связь между ними.

Contrastive Learning — это Metric Learning подход обучения, основанный на измерении расстояния между объектами в пространстве. Обучение происходит следующим образом: вектора похожих объектов "приближаются" друг к другу, а вектора разных "раздвигаются", т.е. мы учим модель проводить более чёткую (контрастную) линию между похожими и непохожими объектами - отсюда и название Contrastive Learning  

Предобучение (pre-training) — это этап обучения модели машинного обучения, на котором модель инициализируется и обучается на большом объеме данных. Целью pre-training'a является получение моделью общих навыков, которые могут быть полезны для широкого круга задач. 

Image Encoder — модель машинного обучения, преобразующая изображения в эмбеддинги. В результате работы Image Encoder'a, изображение становится некой точкой в многомерном пространстве эмбеддингов.

Zero-shot prediction — способность модели машинного обучения решать задачи, на которые она специально не была обучена. Например, классифицировать объекты между классами, которые не были в явном виде представлены в обучающей выборке.

Обучение CLIP происходит на батче из пар "картинка-текст". Каждая картинка проходит через Image Encoder, а каждый текст — через Text Encoder. В результате получаются пары векторных представлений, находящиеся в едином пространстве. 

Далее, вычисляется матрица всех возможных скалярных произведений пар этих векторов, которая в дальнейшем служит для определения степени сходства между каждой парой изображений и текстовых описаний.

Для обучения модели CLIP используется метод максимизации косинусной близости (Cosine Similarity), которая нацелена на уменьшение угла между векторами правильных пар "картинка-текст" и увеличение между неправильными парами.

Косинус угла достигает своего максимального значения (1), когда угол θ = 0 градусов (вектора сонаправлены), и минимального значения (-1), когда угол θ = 180 градусов (вектора противоположно направлены).

Таким образом, скалярные произведения, находящиеся на главной диагонали матрицы, будут максимизированы, поскольку они представляют собой скалярные произведения между правильными парами "картинка-текст". Все остальные скалярные произведения, которые соответствуют неправильным парам, будут минимизированы.

Этот подход позволяет модели научиться точно связывать изображения и тексты, понимая их семантические соответствия.



## Use for zero-shot prediction

Таким образом, чтобы классифицировать изображение, достаточно извлечь векторные представления для этого изображения, а также векторные представления для всех возможных категорий, которые могут быть предположительно классифицированы.

Далее, моделью вычисляются скалярные произведения между эмбеддингом изображения и каждым из эмбеддингов текстовых меток. Эти значения служат мерами сходства, показывающими, насколько каждое из текстовых описаний соответствует изображению.

Наконец, текстовая метка с наибольшим значением сходства выбирается как предсказанная категория для изображения. 

Преимущества CLIP
1. "Обучение без обучения" (zero-shot prediction)
CLIP демонстрирует выдающиеся способности к классификации объектов, которые не были представлены в тренировочном наборе данных. Это означает, что модель может распознавать и классифицировать новые, ранее невиданные объекты, без необходимости дополнительной разметки и обучения на этих конкретных данных. Таким образом, модель снижает необходимость в дорогостоящей и трудоемкой разметке данных.

2. Мультимодальное понимание
CLIP успешно решает широкий спектр задач в компьютерном зрении и NLP благодаря эффективному мультимодальному обучению, что позволяет ему работать с задачами, требующими смешения картинок и текста. Например, классифицировать визуальные запросы пользователей на основе их текстовых описаний (чем мы и займемся в этой задаче). Высокая точность и производительность модели делают её надежным выбором для критически важных приложений и сервисов.

3. Гибкость и универсальность
CLIP может использоваться для решения широкого спектра задач, включая классификацию изображений, генерацию описаний, поиск по изображениям и многое другое. Универсальность модели делает её полезной в различных приложениях и позволяет интегрировать её в различные системы и сервисы.

Недостатки CLIP
1. Подверженность текстовым атрибутам
Недостатком CLIP является подверженность текстовым атрибутам. Модель CLIP может быть сильно подвержена влиянию текстовой информации на изображении, иногда в ущерб визуальной информации. Это приводит к тому, что модель может идентифицировать объект не по его визуальным характеристикам, а по тексту, присутствующему на изображении.
2. Качество данных
CLIP обучался на больших объемах данных из интернета. Не все данные, найденные в интернете, являются качественными или точными, что может повлиять на производительность модели.

3. Производительность в специфических задачах
CLIP может не работать так же хорошо в узкоспециализированных областях или задачах, для которых у неё недостаточно данных во время обучения. Кроме того, модель может плохо работать с языками или диалектами, которые недостаточно хорошо представлены в обучающих данных.



# Задание
В этом задании мы будем использовать модель CLIP-ViT-tiny-large-patch14. Это уменьшенная версия модели CLIP, которая в качестве Image Encoder использует ViT-L/14. 

1. Создайте класс для подготовки изображения
Напишите класс ImageProcessor, который будет отвечать за подготовку изображения для дальнейшей обработки.

В классе должен быть метод prepare_image, который принимает путь к изображению, читает его, кодирует в base64 и возвращает подготовленное изображение.

In [11]:
import os
import base64
from PIL import Image
import io

In [12]:
class ImageProcessor:
    """
    A class for processing images.

    Attributes:
        image_path (str): The path to the image.
    """
    def __init__(self, image_path: str):
        """
        Initialize an ImageProcessor instance.

        Parameters:
            image_path (str): The path to the image.

        Raises:
            ValueError: If the image path is empty.
        """
        if not image_path:
            raise ValueError("Image path is empty." )
        if not os.path.exists(image_path):
            raise ValueError("Image file does not exist at the provided path.")
        self.image_path = image_path 

    def prepare_image(self) -> Image.Image:
        """
        Method to prepare the image for processing.

        Returns:
            Image.Image: The prepared image.
        """
        # try:
        #     # Open the image
        #     with Image.open(self.image_path) as image:
        #         buffered = io.BytesIO()
        #         image.save(buffered, format=image.format) 
        #         image_bytes = buffered.getvalue()
        # 
        #         encoded_image = base64.b64encode(image_bytes).decode("utf-8")
        # 
        #     return encoded_image
        try:
            # Open the image and return it directly
            image = Image.open(self.image_path).convert("RGB")
            return image

        except IOError as e:
            raise IOError(f"Unable to process the image: {e}")

2. Создайте класс для подготовки модели CLIP
Напишите класс ClipModel, который будет отвечать за загрузку и подготовку модели CLIP для классификации.

В классе должен быть метод classify, который принимает данные для классификации и возвращает распределение вероятностей в виде списка. 

In [13]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
import torch
class ClipModel:
    """
    A class for working with a classification model.

    Attributes:
        model (CLIPModel): The CLIP model instance.
        processor (CLIPProcessor): The CLIP processor instance.
    """
    def __init__(self):
        """
        Initialize a ClipModel instance.
        """
        # Load model directly
        self.model = AutoModelForZeroShotImageClassification.from_pretrained("yujiepan/clip-vit-tiny-random-patch14-336")
        self.processor = AutoProcessor.from_pretrained("yujiepan/clip-vit-tiny-random-patch14-336")

    def classify(self, image: Image.Image, intents: list) -> list:
        """
        Method for classifying data.

        Parameters:
            image (Image.Image): The image to classify.
            intents (list): The list of text categories.

        Returns:
            list: The classification result as probability distribution.
        """
        inputs = self.processor(text=intents, images=image, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)

        # Extract scores and normalize them to probabilities
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
        return probs[0].detach().numpy().tolist()

3. Создайте класс для классификации изображения
Напишите класс ImageClassifier, который будет использовать ImageProcessor для подготовки изображения и ClipModel для его классификации.

В классе должен быть метод classify_intent, который принимает путь к изображению, подготавливает его, отправляет его в модель для классификации и возвращает результат.

Метод classify_intent должен возвращать один из трех тегов в виде строки: "payment-problem", "course-problem", "profile-problem".

In [14]:
from typing import List

class ImageClassifier:
    """
    A class for classifying images.

    Attributes:
        clip_model (ClipModel): An object for data classification.
        intents (List[str]): A list of categories for classification.
    """
    def __init__(self):
        """
        Initialize an ImageClassifier instance.
        """
        self.clip_model = ClipModel()  # Initialize the ClipModel
        self.intents = ["payment-problem", "course-problem", "profile-problem"] 

    def classify_intent(self, image_path: str) -> str:
        """
        Method for classifying intent.

        Parameters:
            image_path (str): The path to the image.

        Returns:
            str: The category of intent.
        """
        image = ImageProcessor(image_path)
        prepared_image = image.prepare_image()
        
        probs = self.clip_model.classify(prepared_image,self.intents)
        max_index = probs.index(max(probs))
        
        # Return the corresponding intent
        return self.intents[max_index]


In [15]:
classifier = ImageClassifier()
result = classifier.classify_intent("payment.png") # 'payment-problem'

In [16]:
result

'payment-problem'

payment-problem
